| Pattern       | Category       | Purpose                                                                  |
| ------------- | -------------- | ------------------------------------------------------------------------ |
| **Singleton** | **Creational** | Ensures only one instance of a class exists.                             |
| **Factory** | **Creational** | Creates objects without specifying exact class.                          |
| **Builder** | **Creational** | Builds complex objects step-by-step.                                     |
| **Adapter** | **Structural** | Converts one interface into another expected by the client.              |
| **Strategy** | **Behavioral** | Allows selecting an algorithm/strategy at runtime.                       |
| **Observer** | **Behavioral** | Notifies objects automatically when another object changes.              |
| **State** | **Behavioral** | Allows an object to change its behavior when its internal state changes. |

# Singleton Design Pattern

In [121]:
import logging
from abc import ABC, abstractmethod, ABCMeta
import threading

# Thread-safe Singleton Metaclass
class ThreadSafeSingleton(ABCMeta):
    _instances = {}
    _lock = threading.Lock()

    def __call__(cls, *args, **kwargs):
        with cls._lock:
            if cls not in cls._instances:
                cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

In [122]:
# Interface for Logger
class BaseLogger(ABC):
    @abstractmethod
    def debug(self, message): pass

    @abstractmethod
    def info(self, message): pass

    @abstractmethod
    def warning(self, message): pass

    @abstractmethod
    def error(self, message): pass

In [123]:
import logging

class LoggerSystem(BaseLogger, metaclass=ThreadSafeSingleton):
    """
        LoggerSystem is a class which deals with invoking the logger methods
        Methods :- 
            _get_or_create_logger :- Helps to get or create the logger instance
            debug :- This Functions reads the level 10 logger data
            info :- This Functions reads the level 20 logger data
            error :- This Functions reads the level 40 logger data
            warning :- This Functions reads the level 30 logger data

    """
    def __init__(self):
        self.app_name = 'CodeGeN'
        self.level = logging.DEBUG  
        self.formatter = '[%(asctime)s] %(levelname)s - %(message)s]'
        self._logger = self._get_or_create_logger()
        
    def _get_or_create_logger(self):
        logger = logging.getLogger(self.app_name)
        logger.setLevel(self.level)

        if logger.hasHandlers():
            return logger

        formatter = logging.Formatter(self.formatter)

        # Console Handler
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(formatter)

        # File Handler
        file_handler = logging.FileHandler('app.log', mode='a')
        file_handler.setFormatter(formatter)

        # Add handlers
        logger.addHandler(console_handler)
        logger.addHandler(file_handler)

        return logger
    
    def debug(self, message):
        return self._logger.debug(message)
    def error(self, message):
        return self._logger.error(message)
    def warning(self, message):
        return self._logger.warning(message)
    def info(self, message):
        return self._logger.info(message)

logger =  LoggerSystem()
logger1 =  LoggerSystem()
print(id(logger))
print(id(logger1))

logger.debug("Hi")
logger.info("Hi")
logger.warning("Hi")
logger.error("Hi")



[2025-07-27 15:12:46,030] DEBUG - Hi]
[2025-07-27 15:12:46,034] INFO - Hi]
[2025-07-27 15:12:46,038] WARNING - Hi]
[2025-07-27 15:12:46,040] ERROR - Hi]


2797220710864
2797220710864


In [124]:
## Example -2 Config Manager

In [125]:
import yaml
from abc import ABC, abstractmethod

class BaseConfig(ABC):
    def __init__(self):
        self._config = self._load_config()  # No path passed here
    
    @abstractmethod        
    def _load_config(self, path=None):
        pass

    @abstractmethod
    def get_data(self, parent, child):
        pass


# Assuming ThreadSafeSingleton is a metaclass you already have
class ConfigManager(BaseConfig, metaclass=ThreadSafeSingleton):
    def _load_config(self, path='config/config.yaml'):
        with open(path, 'r') as file:
            return yaml.safe_load(file)
    
    def get_data(self, parent, child):
        parent_data = self._config.get(parent)
        if parent_data is None:
            raise ValueError(f"Invalid parent key: '{parent}'")

        child_data = parent_data.get(child)
        if child_data is None:
            raise ValueError(f"Invalid child key: '{child}' under parent '{parent}'")
        
        return child_data


In [126]:
config_1 = ConfigManager()
config_2 = ConfigManager()
print(id(config_1))
print(id(config_2))
config_1.get_data("logging","level")
config_2.get_data("logging","file")



2797220608912
2797220608912


'logs/app.log'

# Factory Method

In [127]:
from abc import ABC, abstractmethod

# Step 1: Interface
class PaymentGateway(ABC):
    @abstractmethod
    def process_payment(self, amount):
        pass

# Step 2: Concrete Classes
class PayPal(PaymentGateway):
    def process_payment(self, amount):
        print("Processing payment via PayPal:", amount)

class RazoPay(PaymentGateway):
    def process_payment(self, amount):
        print("Processing payment via RazorPay:", amount)

# Step 3: Factory
class PaymentGatewayFactory:
    @staticmethod
    def get_gateway(payment_type: str) -> PaymentGateway:
        payment_type = payment_type.lower()
        if payment_type == 'razopay':
            return RazoPay()
        elif payment_type == 'paypal':
            return PayPal()
        else:
            raise ValueError("Invalid payment type")

gateway = PaymentGatewayFactory.get_gateway("razopay")
gateway.process_payment(1000)

Processing payment via RazorPay: 1000


# Builder Design Pattern
- The Builder Pattern is a creational design pattern used to construct complex objects step-by-step. It separates the construction process from the actual representation of the object.

In [128]:
# 🍕 1. Without Builder Pattern (Simple Instantiation)

In [129]:
class Pizza:
    def __init__(self, dough, sauce, topping):
        self.dough = dough
        self.sauce = sauce
        self.topping = topping

    def __str__(self):
        return f"Pizza with {self.dough} dough, {self.sauce} sauce, and {self.topping} topping."

# Create pizza directly
pizza = Pizza("pan baked", "spicy", "pepperoni")
print(pizza)


Pizza with pan baked dough, spicy sauce, and pepperoni topping.


## Downsides:
* Hard to manage when there are many options or configurations.

* Constructor can get messy if more ingredients are added.

* Changing construction logic means changing the core class.

### 🧩 Components of the Builder Pattern

| Component         | Role & Responsibility                                                               |
|-------------------|--------------------------------------------------------------------------------------|
| **Product**        | The complex object being built (e.g., `Pizza`, `Car`, etc.)                         |
| **Builder Interface** | Defines the abstract methods for building parts of the product                    |
| **Concrete Builder**  | Implements the builder interface and provides specifics for each build step       |
| **Director**       | Orchestrates the steps using the builder to create the product                      |
| **Client**         | Initiates the building process by choosing a builder and optionally a director      |


In [130]:
# 1. 🧱 Product: Pizza
class Pizza:
    def __init__(self):
        self.dough = None
        self.sauce = None
        self.topping = None

    def __str__(self):
        return f"Pizza with {self.dough} dough, {self.sauce} sauce, and {self.topping} topping."


In [131]:
# 2. 🔧 Builder Interface

from abc import ABC, abstractmethod

class PizzaBuilder(ABC):
    @abstractmethod
    def build_dough(self): pass

    @abstractmethod
    def build_sauce(self): pass

    @abstractmethod
    def build_topping(self): pass

    @abstractmethod
    def get_pizza(self): pass


In [132]:
# 3. 🏗️ Concrete Builder

class VegPizzaBuilder(PizzaBuilder):
    def __init__(self):
        self.pizza = Pizza()

    def build_dough(self):
        self.pizza.dough = "whole wheat"

    def build_sauce(self):
        self.pizza.sauce = "herb tomato"

    def build_topping(self):
        self.pizza.topping = "bell peppers, olives, corn"

    def get_pizza(self):
        return self.pizza


class ChickenPizzaBuilder(PizzaBuilder):
    def __init__(self):
        self.pizza = Pizza()

    def build_dough(self):
        self.pizza.dough = "classic"

    def build_sauce(self):
        self.pizza.sauce = "bbq"

    def build_topping(self):
        self.pizza.topping = "grilled chicken, onions"

    def get_pizza(self):
        return self.pizza

class MushroomPizzaBuilder(PizzaBuilder):
    def __init__(self):
        self.pizza = Pizza()

    def build_dough(self):
        self.pizza.dough = "cheese burst"

    def build_sauce(self):
        self.pizza.sauce = "garlic white"

    def build_topping(self):
        self.pizza.topping = "button mushrooms, jalapeños"

    def get_pizza(self):
        return self.pizza


In [133]:
class PizzaChef:
    def __init__(self, builder: PizzaBuilder):
        self.builder = builder

    def make_pizza(self):
        self.builder.build_dough()
        self.builder.build_sauce()
        self.builder.build_topping()
        return self.builder.get_pizza()


In [134]:
for builder in [VegPizzaBuilder(), ChickenPizzaBuilder(), MushroomPizzaBuilder()]:
    chef = PizzaChef(builder)
    pizza = chef.make_pizza()
    print(pizza)


Pizza with whole wheat dough, herb tomato sauce, and bell peppers, olives, corn topping.
Pizza with classic dough, bbq sauce, and grilled chicken, onions topping.
Pizza with cheese burst dough, garlic white sauce, and button mushrooms, jalapeños topping.


In [135]:
# Example - 2
# Notification System

In [153]:
import uuid
# Product
class Notification:
    def __init__(self):
        self.id = uuid.uuid4()
        self.notification_type = None
        self.sender = None
        self.receiver = None
        self.message = None
    
    def __str__(self):
        return f'{self.id}-{self.notification_type}'

In [154]:
# notification builder
from abc import abstractmethod, ABC
class NotificationBuilder(ABC):
    @abstractmethod
    def add_notification_type(self):
        pass
    
    @abstractmethod
    def add_sender(self):
        pass
    
    @abstractmethod
    def add_receiver(self):
        pass

    @abstractmethod
    def add_message(self):
        pass
    
    @abstractmethod
    def get_notification(self):
        pass

In [155]:
# 3. 🏗️ Concrete Builder
class EmailNotification(NotificationBuilder):
    def __init__(self):
        self.notification = Notification()
    def add_notification_type(self):
        self.notification.notification_type = "Email"
    def add_sender(self):
        self.notification.sender = "shanu@gmail.com"
    def add_receiver(self):
        self.notification.receiver = "adari@gmail.com"
    def add_message(self):
        self.notification.message = "Hi Hello through email"
    def get_notification(self):
        return self.notification


class SMSNotification(NotificationBuilder):
    def __init__(self):
        self.notification = Notification()
    def add_notification_type(self):
        self.notification.notification_type = "phone"
    def add_sender(self):
        self.notification.sender = "9999999999999999"
    def add_receiver(self):
        self.notification.receiver = "2222222222222222222"
    def add_message(self):
        self.notification.message = "Hi Hello"
    def get_notification(self):
        return self.notification
    
    

In [156]:
class NotificationOrchestrator:
    def __init__(self, builder):
        self.builder = builder
    def create_notification(self):
        self.builder.add_notification_type()
        self.builder.add_sender()
        self.builder.add_receiver()
        self.builder.add_message()
        return self.builder.get_notification()
        

In [157]:
for notify in [EmailNotification(), SMSNotification()]:
    orchestrator = NotificationOrchestrator(notify)
    result = orchestrator.create_notification()
    print(result)


e8042f24-8f54-40d6-83d2-0aba164ba9ce-Email
9b33941f-5373-4b19-93bb-8849c2fee645-phone


# What is the Adapter Pattern?
The Adapter pattern is a structural design pattern that facilitates the interaction between two interfaces that are incompatible or cannot work together directly. It acts as a bridge, allowing objects with different interfaces to collaborate.
- We have an old method in  a class which users love to use now we have to replace it with another vendor but client does not want to change the way its invoked so now the new methods changes in such a way it adapates to format of old method




## Conceptual Diagram

The diagram you provided illustrates this pattern perfectly.

```text
                 ┌────────────────────┐
                 │    UpiPayment      │◄───────┐  (Target Interface)
                 └────────────────────┘        │
                      ▲                        │
     ┌────────────────┴─────────────┐          │
     │        Adapter Classes        │          │
     │ VisaPaymentAdapter           │           │
     │ MasterCardPaymentAdapter     │           │
     └──────────────────────────────┘           │
             ▲                       ▲          │
             │                       │          │
     ┌───────┴───────┐     ┌─────────┴─────┐    │
     │  VisaPayment  │     │ MasterCardPayment│ │
     └───────────────┘     └──────────────────┘ │
          (Adaptees)                            │
                                                │
                   ┌────────────────────────────┘
                   │
          Client Code Uses: `.make_payment()`

In [7]:
class UpiPayment:  # Target interface
    def make_payment(self, amount):
        print(f"Amount Credited via UPI: ₹{amount}")


# Adaptees
class VisaPayment:
    def pay(self, amount):
        print(f"Amount Credited via Visa: ₹{amount}")


class MasterCardPayment:
    def add_payment(self, amount):
        print(f"Amount Credited via MasterCard: ₹{amount}")


# Adapters
class VisaPaymentAdapter(UpiPayment):
    def __init__(self, visa: VisaPayment):
        self.visa = visa

    def make_payment(self, amount):
        self.visa.pay(amount)


class MasterCardPaymentAdapter(UpiPayment):
    def __init__(self, mastercard: MasterCardPayment):
        self.mastercard = mastercard

    def make_payment(self, amount):
        self.mastercard.add_payment(amount)


# Usage
visa_adapter = VisaPaymentAdapter(VisaPayment())
visa_adapter.make_payment(500)

mastercard_adapter = MasterCardPaymentAdapter(MasterCardPayment())
mastercard_adapter.make_payment(1000)


Amount Credited via Visa: ₹500
Amount Credited via MasterCard: ₹1000
